<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/TwADR-L/Code/twadr_pubmed_tweet_overlap_20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

In [0]:
base_dir = 'medical-concept-normalization'

In [0]:
if os.path.exists('medical-concept-normalization'):
  print('folder exist')
else:
  os.mkdir(base_dir)

In [0]:
if os.path.exists('medical-concept-normalization/data_collection'):
  print('folder exist')
else:
  os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: course-v3.fast.ai


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [0]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [0]:
path = Path('medical-concept-normalization/data_collection')

In [0]:
# datasets

train_csv = 'https://raw.githubusercontent.com/unt-iialab/medical-concept-normalization/master/TwADR-L/Experiment%20(Change%20test%20dataset)/Training%20set/tward_training_folder_0.csv'
valid_csv = 'https://raw.githubusercontent.com/unt-iialab/medical-concept-normalization/master/TwADR-L/Experiment%20(Change%20test%20dataset)/Validation%20set/tward_validation_folder_0.csv'
test_csv = 'https://raw.githubusercontent.com/unt-iialab/medical-concept-normalization/master/TwADR-L/Experiment%20(Change%20test%20dataset)/Testing%20set/tward_testing_overlap_20.csv'
fine_tuning_data = 'fine_tuning_language_model_pubmed_bigtweet_mixed.csv'

In [10]:
# training data set
train = pd.read_csv(train_csv, header=None)
print(train.head())

# validating data set
valid = pd.read_csv(valid_csv,header=None)
# valid.head()

# testing data set
test = pd.read_csv(test_csv,header=None)
# test.head()

      0                      1
0  1945    Adrenergic syndrome
1  1586  Hearing Loss, Partial
2   580          Typhoid Fever
3   784      Carcinoma of lung
4   660              Pregnancy


In [12]:
# Creating DataBunch for language modelling
data_lm = TextLMDataBunch.from_csv(path, fine_tuning_data)

In [0]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [14]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [15]:
data_clas.add_test(test,label=0)

In [0]:
data_lm.save()
data_clas.save()

In [17]:
data_lm.train_ds.vocab.itos

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 'the',
 'of',
 'and',
 'in',
 'to',
 'a',
 'for',
 'with',
 'diclofenac',
 'was',
 'were',
 'on',
 'is',
 'rt',
 'sodium',
 'by',
 'that',
 'as',
 'at',
 'patients',
 'or',
 'this',
 'from',
 'drug',
 'you',
 'i',
 'group',
 'be',
 'mg',
 'study',
 'disease',
 'are',
 'treatment',
 'pain',
 'after',
 'an',
 'not',
 'amp',
 'it',
 'have',
 'p',
 'release',
 'gluten',
 'we',
 'free',
 'than',
 'all',
 'effect',
 'no',
 'new',
 'compared',
 'effects',
 'using',
 'more',
 'groups',
 'inflammatory',
 'two',
 'your',
 'about',
 'its',
 'both',
 'between',
 'our',
 'time',
 'significantly',
 'h',
 'anti',
 'drugs',
 'significant',
 'my',
 'these',
 'out',
 'day',
 'but',
 'used',
 'use',
 'voltaren',
 'which',
 'can',
 'dose',
 'also',
 'potassium',
 'has',
 'placebo',
 'results',
 'what',
 'one',
 'glutenfree',
 'do',
 'there',
 's',
 'efficacy',
 'nt',
 'may',
 'days',
 'celiac',
 'ds',
 'treated',
 '

In [18]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [19]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [20]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,5.188019,4.615873,0.251321,21:50


In [21]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.434154,4.271044,0.285370,21:49


In [22]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.123131,4.231622,0.289462,21:50


In [23]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.076591,4.200237,0.292773,21:40


In [0]:
learn.save_encoder('tuning_pubmed_bigtweet_overlap_0_first')

In [25]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('tuning_pubmed_bigtweet_overlap_0_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,7.103562,6.363090,0.130435,00:52


In [26]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,6.210863,5.227492,0.165217,00:51


In [27]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.744867,4.817636,0.173913,01:08


In [28]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.478954,4.497169,0.200000,01:04
1,5.291551,4.397794,0.165217,01:09


In [29]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.086654,4.228884,0.226087,01:08
1,4.927964,4.174865,0.226087,01:05


In [30]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.785192,4.023246,0.269565,01:07
1,4.640033,3.981030,0.269565,01:04


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.462239,3.989191,0.234783,01:07
1,4.354757,3.915006,0.260870,01:03


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.234307,3.908525,0.226087,01:08
1,4.084385,3.905091,0.269565,01:02


In [33]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.910795,3.915743,0.252174,01:06
1,3.872776,3.927238,0.269565,01:05
2,3.733199,3.842450,0.313043,01:06
3,3.592998,3.839024,0.304348,01:05


In [34]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.433017,3.813226,0.304348,01:06
1,3.399888,3.863372,0.260870,01:03
2,3.240802,3.822291,0.278261,01:08
3,3.117996,3.801852,0.278261,01:09


In [35]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.990614,3.840260,0.295652,01:04
1,2.984731,3.920793,0.252174,01:05
2,2.855926,3.857522,0.295652,01:07
3,2.730137,3.841486,0.304348,01:06


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.629502,3.843346,0.304348,01:03
1,2.643148,3.853421,0.286957,01:07
2,2.536096,3.894631,0.278261,01:06
3,2.424392,3.875277,0.313043,01:04


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.332278,3.877383,0.313043,01:04
1,2.333051,3.867569,0.295652,01:08
2,2.318274,3.956567,0.295652,01:08
3,2.222308,3.923008,0.295652,01:02


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.137910,3.940499,0.313043,01:04
1,2.182122,4.112158,0.286957,01:05
2,2.105710,4.099386,0.252174,01:08
3,2.025974,4.084437,0.269565,01:08


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.961349,4.032089,0.304348,01:06
1,1.968117,4.124346,0.286957,01:07
2,1.923880,4.076173,0.304348,01:07
3,1.862399,4.064598,0.313043,01:06


In [40]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.839153,4.205594,0.252174,01:08
1,1.819650,4.188605,0.252174,01:08


In [41]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.876571,4.283625,0.278261,01:06
1,1.832585,4.224254,0.278261,01:02


In [42]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.820507,4.227181,0.252174,01:10


In [0]:
# save the best model

learn.save_encoder('tuning_pubmed_bigtweet_overlap_0_last')

# Part three: Predict on the test dataset

In [44]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [45]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

180
180
[2099, 2099, 170, 1247, 2099, 1639, 2099, 803, 135, 1411, 135, 2099, 854, 854, 860, 1136, 854, 210, 965, 1072, 106, 803, 2099, 2099, 1802, 171, 1077, 2099, 1698, 1348, 491, 2099, 135, 1576, 2088, 1136, 2099, 2099, 2065, 1411, 2099, 1407, 1932, 171, 106, 1935, 1307, 621, 1307, 854, 803, 574, 2099, 1723, 135, 1546, 767, 1307, 854, 1308, 1038, 2065, 1932, 1932, 2099, 2099, 2099, 309, 1956, 860, 2099, 1811, 309, 52, 1570, 505, 2099, 2099, 978, 574, 724, 1038, 92, 309, 909, 1323, 135, 2195, 1307, 803, 230, 1811, 135, 2099, 621, 1307, 1570, 798, 378, 2099, 491, 2099, 931, 860, 854, 1034, 1478, 854, 309, 700, 1411, 1136, 135, 2099, 2046, 2099, 2099, 1956, 574, 2138, 1811, 1698, 2145, 2065, 1677, 135, 2099, 854, 1534, 2099, 170, 2099, 574, 309, 2099, 135, 2099, 1472, 2099, 2099, 1308, 621, 1773, 170, 702, 1247, 1034, 2099, 2099, 1811, 983, 189, 840, 170, 441, 2046, 2099, 2099, 700, 2099, 1392, 170, 135, 1308, 1376, 2046, 135, 1652, 2099, 170, 2099, 1576, 249, 89, 441, 909, 847, 2099, 2

In [46]:
correct_num = 0

for i in range(0, len(standard_list)):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

43
0.2388888888888889
